# Create Water Level Snapshot - pick time

**What this notebook does** This notebook creates a snapshot of latest water body conditions, appending the dam fill level as an attribute in a shapefile. This code needs to be run manually after new data has been ingested into DEA, and after the .txt files for each water bosy have been updated.

**Required inputs** 
- A shapefile to be appended with the latest data (`Shapefile_to_append`)
- A folder where all the `.txt files` are located from which the snapshot will be created (`Timeseries_dir`)
- The date for analysis needs to be updated below (`DateToExtractStr`)

**Date** December 2018

**Author** Claire Krause

In [1]:
import subprocess
import glob
import geopandas as gp
import re
import pandas as pd
from dateutil.relativedelta import relativedelta

## Set up the file paths, date to be extracted and output file names

`Timeseries_dir` is the folder where all the .txt files are located from which the snapshot will be created

`Shapefile_to_append` is the shapefile which the snapshot will be added to as an attribute

In [2]:
Timeseries_dir = '/g/data/r78/cek156/dea-notebooks/Dams/Dams2000to2018/Timeseries_new/'
Shapefile_to_append = '/g/data/r78/cek156/dea-notebooks/Dams/Dams2000to2018/AllNSW2000to201810LatestSnapshot.shp'

SnapshotShp = gp.read_file(Shapefile_to_append)

### Choose when you want the snapshot for

Note that there is a filter built in here that excludes observations that are more than a month from the chosen date.

In [3]:
DateToExtractStr = '20181207'
DateToExtract = pd.to_datetime(DateToExtractStr, format='%Y%m%d')

# Create filter range which only includes observations of +/- 45 days
DateFilterA = DateToExtract + relativedelta(days=-45)
DateFilterB = DateToExtract + relativedelta(days=45)

## Get a list of all of the files in the folder to loop through

In [4]:
TimeSeriesFiles = glob.glob(f'{Timeseries_dir}*.csv')

## Loop through and extract the relevant date from all the .txt files

In [5]:
for file in TimeSeriesFiles:
    # Get the ID
    NameComponents = re.split('\.|/', file)  # Splitting on '.' and '/'
    PolyID = NameComponents[-2]
    PolyID = int(PolyID)
    try:
        AllObs = pd.read_csv(file, parse_dates=[
                             'Observation Date'], index_col='Observation Date')
        x = AllObs.iloc[AllObs.index.get_loc(DateToExtract, method='nearest')]
        if(x.name > DateFilterA and x.name < DateFilterB):
            ObsDate = str(x.name)
            Pct = float(x['Wet pixel percentage'])
            FindPolyIndex = SnapshotShp.where(
                SnapshotShp['ID'] == PolyID).dropna(how='all').index.values[0]
            SnapshotShp.loc[(FindPolyIndex, f'{DateToExtractStr}')] = ObsDate
            SnapshotShp.loc[(FindPolyIndex, f'Pct{DateToExtractStr}')] = Pct
        else:
            print(x.name, 'is out of snapshot range.')
    except:
        print(f'Bad {PolyID}')
        continue

2018-08-23 23:42:30 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2018-04-17 23:45:36 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2007-09-02 23:37:18 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2007-09-02 23:37:18 is out of snapshot range.
2018-10-02 23:42:40 is out of snapshot range.
2018-09-18 00:25:24 is out of snap

2018-09-18 00:25:48 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-10-02 23:42:40 is out of snapshot range.
2018-10-02 23:42:16 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-09-16 23:42:08 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-10-04 00:25:56 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-03-06 23:55:43 is out of snapshot range.
2008-02-16 23:41:38 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2018-09-16 23:42:08 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:48 is out of snap

2011-11-16 23:31:56 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-09-30 23:55:00 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2008-02-09 23:35:11 is out of snapshot range.
2018-09-09 23:35:56 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2007-08-17 23:37:51 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-10-02 23:42:16 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2018-10-02 23:42:40 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2018-10-04 00:25:32 is out of snap

2018-09-18 00:25:24 is out of snapshot range.
2018-09-25 00:31:38 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2011-10-15 23:32:08 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2018-09-30 23:54:37 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-18 00:25:48 is out of snap

2007-09-02 23:37:18 is out of snapshot range.
2018-10-02 23:42:16 is out of snapshot range.
2018-09-16 23:42:08 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2008-02-09 23:35:35 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-30 23:55:00 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-10-04 00:25:56 is out of snapshot range.
2018-09-18 00:25:48 is out of snap

2018-09-18 00:25:24 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-04-17 23:44:48 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-09-09 23:35:56 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-08-23 23:42:30 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2008-02-09 23:35:11 is out of snapshot range.
2018-10-04 00:25:32 is out of snap

2018-08-23 23:42:30 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-16 23:42:08 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-30 23:55:00 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2008-02-09 23:35:35 is out of snapshot range.
2018-02-10 23:58:01 is out of snapshot range.
2008-03-12 23:34:11 is out of snapshot range.
2007-08-17 23:37:28 is out of snapshot range.
2018-09-16 23:42:08 is out of snap

2018-10-02 23:42:40 is out of snapshot range.
2018-09-22 00:02:42 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2008-02-09 23:34:47 is out of snapshot range.
2018-05-03 23:44:30 is out of snapshot range.
2018-09-16 23:42:08 is out of snapshot range.
2008-02-09 23:35:11 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2011-11-16 23:31:56 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2007-09-02 23:37:42 is out of snapshot range.
2018-10-02 23:42:40 is out of snapshot range.
2018-09-18 00:25:24 is out of snapshot range.
2018-08-23 23:43:18 is out of snapshot range.
2008-01-24 23:35:26 is out of snapshot range.
2018-10-04 00:27:08 is out of snapshot range.
2018-09-18 00:25:48 is out of snapshot range.
2018-10-04 00:25:32 is out of snapshot range.
2018-10-02 23:42:16 is out of snap

## Write out the appended shapefile

In [6]:
schema = gp.io.file.infer_schema(SnapshotShp)
schema['properties'][f'{DateToExtractStr}'] = 'str'
schema['properties'][f'Pct{DateToExtractStr}'] = 'float' 
SnapshotShp.to_file(Shapefile_to_append, schema = schema)